MIDI DATA PROCESSING

Import MIDI processing libraries

In [2]:
import mido

Try testing out some note and outputs

In [3]:
# Create note
msg = mido.Message('note_on', note=60)
# Create output stream to play note
outport = mido.open_output()
# Send the note
outport.send(msg)

If a note output, congratulations! Things are working!
If not there's probably some internal MIDI issues you should look @

Moving on, try to load in a file

In [7]:
# Import MIDI file
mfile = mido.MidiFile('Aha_-_Take_On_Me.mid')

# Get messages
for i, track in enumerate(mfile.tracks):
    print('Track {}: {}'.format(i, track.name))
    #for msg in track:
    #    print(msg)

Track 0: 
Track 1: Slap Bass 2         
Track 2: Elec Guitar 1       
Track 3: String Sect 3       
Track 4: Doctor Solo         
Track 5: Harpsichord 2       
Track 6: Syn Brass 1         
Track 7: Chorale             
Track 8: Percussion          
Track 9: Syn Brass 1         
Track 10:                     
Track 11:    "Take On Me"     
